In [60]:
import pandas as pd
import os

In [2]:
matches = pd.read_csv('../data/everything_before_models/matches2.csv')

In [3]:
# Check for missing values in the DataFrame
missing_values = matches.isnull().sum()

# Filter columns with missing values
columns_with_nan = missing_values[missing_values > 0]

# Display the columns and their corresponding NaN counts
print("Columns with NaN values and their counts:")
print(columns_with_nan)

Columns with NaN values and their counts:
player1_bet_odds    12
player2_bet_odds    12
dtype: int64


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
list(matches.columns)

['outdoor',
 'player1_bet_odds',
 'player2_bet_odds',
 'player1_Sets_Before',
 'player1_Gems_Before',
 'player2_Sets_Before',
 'player2_Gems_Before',
 'tournament_level',
 'player1_right_handed',
 'player1_age',
 'player2_right_handed',
 'player2_age',
 'best_of',
 'player1_rank',
 'player2_rank',
 'player1_entry_LL',
 'player1_entry_Q',
 'player1_entry_WC',
 'player2_entry_LL',
 'player2_entry_Q',
 'player2_entry_WC',
 'player1_is_seeded',
 'player2_is_seeded',
 'player1_h2h_wins',
 'player2_h2h_wins',
 'player1_home',
 'player2_home',
 'Round_Num',
 'player1_win_pct_last_10',
 'player2_win_pct_last_10',
 'player1_win_pct_last_10_surface',
 'player2_win_pct_last_10_surface',
 'player1_elo',
 'player2_elo',
 'player1_surface_elo',
 'player2_surface_elo',
 'player1_blended_elo',
 'player2_blended_elo',
 'num_CO_matches',
 'player1_CO_1st_serve_in_pct_avg',
 'player2_CO_1st_serve_in_pct_avg',
 'player1_1st_serve_in_pct_avg',
 'player2_1st_serve_in_pct_avg',
 'player1_CO_1st_serve_win_pct

In [6]:
player1_cols = [col for col in matches.columns if col.startswith('player1_') and "bet" not in col]
player2_cols = [col for col in matches.columns if col.startswith('player2_') and "bet" not in col]
env_cols = [col for col in matches.columns if col not in player1_cols and col not in player2_cols and "diff" not in col and "CO" not in col and "target" not in col and "bet" not in col]
target_col = 'target'
player1_bet = 'player1_bet_odds'
player2_bet = 'player2_bet_odds'
player1_cols

['player1_Sets_Before',
 'player1_Gems_Before',
 'player1_right_handed',
 'player1_age',
 'player1_rank',
 'player1_entry_LL',
 'player1_entry_Q',
 'player1_entry_WC',
 'player1_is_seeded',
 'player1_h2h_wins',
 'player1_home',
 'player1_win_pct_last_10',
 'player1_win_pct_last_10_surface',
 'player1_elo',
 'player1_surface_elo',
 'player1_blended_elo',
 'player1_CO_1st_serve_in_pct_avg',
 'player1_1st_serve_in_pct_avg',
 'player1_CO_1st_serve_win_pct_avg',
 'player1_1st_serve_win_pct_avg',
 'player1_CO_2nd_serve_win_pct_avg',
 'player1_2nd_serve_win_pct_avg',
 'player1_CO_serve_games_win_pct_avg',
 'player1_serve_games_win_pct_avg',
 'player1_CO_ace_avg',
 'player1_ace_avg',
 'player1_CO_df_avg',
 'player1_df_avg',
 'player1_CO_1st_serve_return_win_pct_avg',
 'player1_1st_serve_return_win_pct_avg',
 'player1_fatigue_score',
 'player1_tournament_wins_before',
 'player1_tournament_losses_before']

In [7]:
player1_features = matches[player1_cols].values  # Convert to numpy array
player2_features = matches[player2_cols].values
env_features = matches[env_cols].values
match_outcomes = matches[target_col].values  # Target (1 for player1 win, 0 for player2 win)
player1_bet_odds = matches[player1_bet].values
player2_bet_odds = matches[player2_bet].values

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
player1_features = scaler.fit_transform(player1_features)
player2_features = scaler.transform(player2_features)  
env_features = scaler.fit_transform(env_features)

In [9]:
player1_features

array([[0.10526316, 0.08571429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10526316, 0.08571429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10526316, 0.08571429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.42105263, 0.38571429, 0.        , ..., 0.34354583, 0.08823529,
        0.        ],
       [0.47368421, 0.28571429, 1.        , ..., 0.40908869, 0.11764706,
        0.        ],
       [0.52631579, 0.45714286, 0.        , ..., 0.4261071 , 0.11764706,
        0.        ]])

In [10]:
import torch

player1_features = torch.tensor(player1_features, dtype=torch.float32)
player2_features = torch.tensor(player2_features, dtype=torch.float32)
env_features = torch.tensor(env_features, dtype=torch.float32)
match_outcomes = torch.tensor(match_outcomes, dtype=torch.float32).unsqueeze(1)  # Add dimension for compatibility
player1_bet_odds = torch.tensor(player1_bet_odds, dtype=torch.float32).unsqueeze(1)
player2_bet_odds = torch.tensor(player2_bet_odds, dtype=torch.float32).unsqueeze(1)

In [11]:
from sklearn.model_selection import train_test_split

player1_train, player1_test, player2_train, player2_test, env_train, env_test, y_train, y_test, player1_bet_odds_train, player1_bet_odds_test, player2_bet_odds_train, player2_bet_odds_test = train_test_split(
    player1_features, player2_features, env_features, match_outcomes, player1_bet_odds, player2_bet_odds, test_size=0.2, random_state=42, stratify=match_outcomes
)
player1_train, player1_val, player2_train, player2_val, env_train, env_val, y_train, y_val, player1_bet_odds_train, player1_bet_odds_val, player2_bet_odds_train, player2_bet_odds_val = train_test_split(
    player1_train, player2_train, env_train, y_train, player1_bet_odds_train, player2_bet_odds_train, test_size=0.2, random_state=42, stratify=y_train
)
y_val.unique(return_counts=True)

(tensor([0., 1.]), tensor([1129, 1129]))

In [12]:
class PlayerEmbeddingNet(nn.Module):
    def __init__(self, player_feature_size, embedding_size, dropout, hidden_sizes):
        super(PlayerEmbeddingNet, self).__init__()
        
        # Dynamically build the layers
        layers = []
        input_size = player_feature_size
        
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))  # Batch normalization
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))  # Regularization
            input_size = hidden_size
        
        # Add the final layer to project to the embedding size
        layers.append(nn.Linear(input_size, embedding_size))
        layers.append(nn.BatchNorm1d(embedding_size))  # Normalize embeddings
        
        # Define the network
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)  # Output: player embedding

In [13]:
class SymmetricNNWithEnvInteraction(nn.Module):
    def __init__(self, player_feature_size, env_feature_size, embedding_size, hidden_sizes, dropout, embedding_hidden_sizes=[32]):
        """
        Args:
            player_feature_size (int): Number of features for each player.
            env_feature_size (int): Number of environmental features.
            embedding_size (int): Size of the player embedding.
            hidden_sizes (list[int]): List of hidden layer sizes. Each entry adds one hidden layer.
        """
        super(SymmetricNNWithEnvInteraction, self).__init__()
        
        # Shared subnetwork for player stats
        self.shared_player_net = PlayerEmbeddingNet(player_feature_size, embedding_size, dropout, embedding_hidden_sizes)
        
        # Dynamically build the final network
        layers = []
        input_size = embedding_size * 2 + env_feature_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))  # Hidden layer
            layers.append(nn.ReLU())                          # Activation
            layers.append(nn.Dropout(dropout))                    # Dropout
            input_size = hidden_size  # Update input size for the next layer
        
        # Output layer
        layers.append(nn.Linear(input_size, 1))  # Final layer
        layers.append(nn.Sigmoid())             # Output probability
        
        self.final_net = nn.Sequential(*layers)

    def forward(self, player1_features, player2_features, env_features):
        # Generate embeddings for both players
        player1_emb = self.shared_player_net(player1_features)
        player2_emb = self.shared_player_net(player2_features)
        
        # Combine embeddings symmetrically
        combined_embedding = torch.cat([
            player1_emb - player2_emb,
            torch.abs(player1_emb - player2_emb)
        ], dim=1)
        
        # Concatenate with environmental features
        final_input = torch.cat([combined_embedding, env_features], dim=1)
        
        # Pass through the dynamically built final network
        return self.final_net(final_input)


In [14]:
def calculate_kelly_fraction(p, odds):
    b = odds - 1  # Adjust odds to represent decimal profit
    f = (p * b - (1 - p)) / b
    return max(f, 0)  # Only bet if f > 0

In [54]:
def calculate_roi(predictions, avgW, avgL, y_true, max_bet=1.0):
    total_bet = 0
    net_profit = 0

    flat_predictions = [item[0] for item in predictions]  # Flatten predictions
    flat_avgW = [item[0] for item in avgW]               # Flatten avgW
    flat_avgL = [item[0] for item in avgL]               # Flatten avgL
    flat_y_true = [item[0] for item in y_true]           # Flatten y_true
    
    for prob, odds_w, odds_l, actual in zip(flat_predictions, flat_avgW, flat_avgL, flat_y_true):
        # Your logic here
        if odds_w is None or odds_l is None:
            continue
        # Decide the bet
        if prob >= 0.5:  # Predicted winner
            odds = odds_w
            kelly_fraction = calculate_kelly_fraction(prob, odds)
            outcome = 1 if actual == 1 else -1
        else:  # Predicted loser
            odds = odds_l
            kelly_fraction = calculate_kelly_fraction(1 - prob, odds)
            outcome = 1 if actual == 0 else -1

        # Fixed max bet
        bet_amount = kelly_fraction * max_bet
        if bet_amount > 0:
            total_bet += bet_amount
            if outcome == 1:  # Bet won
                net_profit += bet_amount * (odds - 1)
            else:  # Bet lost
                net_profit -= bet_amount
    roi = net_profit / total_bet if total_bet > 0 else 0
    print(net_profit, total_bet)
    return roi


In [55]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch
import random
import numpy as np
# Set random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# Best Configuration: {'batch_size': 64, 'dropout': np.float64(0.25476497662909336), 'hidden_layer_sizes': [64, 32, 16], 'learning_rate': np.float64(0.008841074088520707), 'optimizer': 'Adam', 'weight_decay': np.float64(5.315656057158321e-06)}
# Best Validation Accuracy: 0.6678
# Create DataLoader for training

train_dataset = TensorDataset(player1_train, player2_train, env_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Validation DataLoader
val_dataset = TensorDataset(player1_val, player2_val, env_val, y_val, player1_bet_odds_val, player2_bet_odds_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Initialize the model
model = SymmetricNNWithEnvInteraction(
    player_feature_size=player1_train.shape[1],
    env_feature_size=env_train.shape[1],
    embedding_size=8,   
    hidden_sizes=[64, 32, 16],
    dropout=np.float64(0.25476497662909336)
)

# Define weight initialization
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

model.apply(initialize_weights)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=np.float64(0.008841074088520707),  weight_decay=np.float64(5.315656057158321e-06))
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss

    
    
 # Training loop
best_val_accuracy = 0.0
for epoch in range(20):
    model.train()
    epoch_loss = 0.0
    for p1, p2, env, labels in train_loader:
        optimizer.zero_grad()
        predictions = model(p1, p2, env)
        loss = criterion(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_predictions = []
    val_y_true = []
    val_avgW = []
    val_avgL = []

    with torch.no_grad():
        for p1, p2, env, labels, avgW, avgL in val_loader:  # Add avgW, avgL in validation data
            predictions = model(p1, p2, env)
            loss = criterion(predictions, labels)
            val_loss += loss.item()
            # Collect predictions and true labels for ROI calculation
            val_predictions.extend(predictions.tolist())
            val_y_true.extend(labels.tolist())
            val_avgW.extend(avgW.tolist())
            val_avgL.extend(avgL.tolist())
    val_loss /= len(val_loader)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for p1, p2, env, labels, a, b in val_loader:
            predictions = model(p1, p2, env)
            predicted = (predictions > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print("val accuracy: ", val_accuracy)
    # Calculate validation ROI
    roi = calculate_roi(val_predictions, val_avgW, val_avgL, val_y_true, max_bet=1.0)

    print(f"Epoch {epoch + 1}: Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, ROI: {roi:.4%}")

val accuracy:  0.6470327723649247
-15.990513424085915 178.95545921770204
Epoch 1: Training Loss: 0.6589, Validation Loss: 0.6321, ROI: -8.9355%
val accuracy:  0.6488042515500443
-11.343732324939086 115.41389698220877
Epoch 2: Training Loss: 0.6425, Validation Loss: 0.6332, ROI: -9.8287%
val accuracy:  0.6545615589016829
-4.010992377984639 106.67111915191396
Epoch 3: Training Loss: 0.6389, Validation Loss: 0.6273, ROI: -3.7601%
val accuracy:  0.6594331266607617
-2.1696839248699744 105.42496531911702
Epoch 4: Training Loss: 0.6357, Validation Loss: 0.6290, ROI: -2.0580%
val accuracy:  0.6567759078830824
-10.278599192652774 150.7125120725469
Epoch 5: Training Loss: 0.6351, Validation Loss: 0.6237, ROI: -6.8200%
val accuracy:  0.6479185119574845
-16.001095485373718 195.46743102368825
Epoch 6: Training Loss: 0.6343, Validation Loss: 0.6277, ROI: -8.1861%
val accuracy:  0.6545615589016829
-5.343244164740524 141.72135955186198
Epoch 7: Training Loss: 0.6318, Validation Loss: 0.6259, ROI: -3.7

In [61]:
from sklearn.model_selection import ParameterSampler
import scipy.stats as stats
set_seed(42)

# Parameter grid for random search
param_grid = {
    'learning_rate': stats.loguniform(1e-5, 1e-2),
    'dropout': stats.uniform(0.1, 0.6),
    'hidden_layer_sizes': [
        [128], [256, 128], [256, 128, 64], [512, 256, 128], [64], [32], [16, 8], [64, 32], [64, 64], [64, 32, 16], [64, 32, 32, 16], [256, 128, 64, 32], [512, 256, 128, 64]
    ],
    'batch_size': [32, 64, 128],
    'weight_decay': stats.loguniform(1e-6, 1e-3),
    'optimizer': ['Adam', 'SGD', 'RMSprop']
}

# Generate random hyperparameter samples
n_samples = 200
random_params = list(ParameterSampler(param_grid, n_iter=n_samples, random_state=42))

# Training and evaluation function
def train_and_evaluate(params):
    print(f"Testing configuration: {params}")
    
    # Extract parameters
    learning_rate = params['learning_rate']
    dropout = params['dropout']
    hidden_layer_sizes = params['hidden_layer_sizes']
    batch_size = params['batch_size']
    weight_decay = params['weight_decay']
    optimizer_choice = params['optimizer']
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Model
    model = SymmetricNNWithEnvInteraction(
        player_feature_size=player1_train.shape[1],
        env_feature_size=env_train.shape[1],
        embedding_size=8,
        hidden_sizes=hidden_layer_sizes,
        dropout=dropout
    )
    
    # Initialize weights
    def initialize_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.zeros_(m.bias)
    model.apply(initialize_weights)

    # Optimizer
    if optimizer_choice == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_choice == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_choice == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


    # Loss function
    criterion = nn.BCELoss()

     # Training loop
    for epoch in range(20):
        model.train()
        for p1, p2, env, labels in train_loader:
            optimizer.zero_grad()
            predictions = model(p1, p2, env)
            loss = criterion(predictions, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
    # Validation loop to calculate ROI
    model.eval()
    val_predictions = []
    val_y_true = []
    val_avgW = []
    val_avgL = []

    with torch.no_grad():
        for p1, p2, env, labels, avgW, avgL in val_loader:
            predictions = model(p1, p2, env)
            val_predictions.extend(predictions.tolist())
            val_y_true.extend(labels.tolist())
            val_avgW.extend(avgW.tolist())
            val_avgL.extend(avgL.tolist())

    # Calculate ROI
    roi = calculate_roi(val_predictions, val_avgW, val_avgL, val_y_true, max_bet=1.0)
    print(f"Validation ROI for this configuration: {roi:.4%}")
    return roi, model

# Run random search
best_params = None
best_val_roi = float('-inf')  # Start with negative infinity for maximization
best_model = None

for params in random_params:
    val_roi, model = train_and_evaluate(params)
    if val_roi > best_val_roi:
        best_val_roi = val_roi
        best_params = params
        best_model = model

# Save the best model weights and configuration
if best_model is not None:
    torch.save(best_model.state_dict(), os.path.join("best_models", "best_model_weights.pth"))
    with open(os.path.join("best_models", "best_config.txt"), "w") as f:
        f.write(f"Best Configuration:\n{best_params}\n")
        f.write(f"Best Validation ROI: {best_val_roi:.4%}\n")

print(f"Best Configuration: {best_params}")
print(f"Best Validation ROI: {best_val_roi:.4%}")

Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.5779257921161397), 'hidden_layer_sizes': [64, 32, 32, 16], 'learning_rate': np.float64(0.0021830968390524606), 'optimizer': 'Adam', 'weight_decay': np.float64(2.9380279387035354e-06)}
-0.3563399057850133 71.57519205714021
Validation ROI for this configuration: -0.4979%
Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.15998494949080172), 'hidden_layer_sizes': [64, 32, 32, 16], 'learning_rate': np.float64(0.003967605077052989), 'optimizer': 'RMSprop', 'weight_decay': np.float64(1.1527987128232402e-06)}
-10.762784995292835 117.74114308648157
Validation ROI for this configuration: -9.1411%
Testing configuration: {'batch_size': 64, 'dropout': np.float64(0.5331992633600948), 'hidden_layer_sizes': [32], 'learning_rate': np.float64(1.0053940194693018e-05), 'optimizer': 'Adam', 'weight_decay': np.float64(7.119418600172983e-05)}
-29.669320264701142 401.01249282786273
Validation ROI for this configuration: -7.398

In [63]:
from sklearn.model_selection import ParameterSampler
import scipy.stats as stats
set_seed(42)

# Parameter grid for random search
param_grid = {
    'learning_rate': stats.loguniform(1e-4, 1e-2),
    'dropout': stats.uniform(0.2, 0.4),
    'hidden_layer_sizes': [
        [128], [256, 128], [64], [32], [16, 8], [64, 32], [64, 64], [64, 32, 16], [32,32], [128,128], [64,16]
    ],
    'batch_size': [32, 64, 128],
    'weight_decay': stats.loguniform(1e-6, 1e-5),
    'optimizer': ['Adam', 'SGD'],
    'embedding_size': [4,8,16],
    'embedding_hidden_layer_sizes': [[128], [256, 128], [64], [32], [16, 8], [64, 32], [16]]
}

# Generate random hyperparameter samples
n_samples = 200
random_params = list(ParameterSampler(param_grid, n_iter=n_samples, random_state=42))

# Training and evaluation function
def train_and_evaluate(params):
    print(f"Testing configuration: {params}")
    
    # Extract parameters
    learning_rate = params['learning_rate']
    dropout = params['dropout']
    hidden_layer_sizes = params['hidden_layer_sizes']
    batch_size = params['batch_size']
    weight_decay = params['weight_decay']
    optimizer_choice = params['optimizer']
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Model
    model = SymmetricNNWithEnvInteraction(
        player_feature_size=player1_train.shape[1],
        env_feature_size=env_train.shape[1],
        embedding_size=8,
        hidden_sizes=hidden_layer_sizes,
        dropout=dropout
    )
    
    # Initialize weights
    def initialize_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.zeros_(m.bias)
    model.apply(initialize_weights)

    # Optimizer
    if optimizer_choice == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_choice == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_choice == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


    # Loss function
    criterion = nn.BCELoss()

     # Training loop
    for epoch in range(20):
        model.train()
        for p1, p2, env, labels in train_loader:
            optimizer.zero_grad()
            predictions = model(p1, p2, env)
            loss = criterion(predictions, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
    # Validation loop to calculate ROI
    model.eval()
    val_predictions = []
    val_y_true = []
    val_avgW = []
    val_avgL = []

    with torch.no_grad():
        for p1, p2, env, labels, avgW, avgL in val_loader:
            predictions = model(p1, p2, env)
            val_predictions.extend(predictions.tolist())
            val_y_true.extend(labels.tolist())
            val_avgW.extend(avgW.tolist())
            val_avgL.extend(avgL.tolist())

    # Calculate ROI
    roi = calculate_roi(val_predictions, val_avgW, val_avgL, val_y_true, max_bet=1.0)
    print(f"Validation ROI for this configuration: {roi:.4%}")
    return roi, model

# Run random search
best_params = None
best_val_roi = float('-inf')  # Start with negative infinity for maximization
best_model = None

for params in random_params:
    val_roi, model = train_and_evaluate(params)
    if val_roi > best_val_roi:
        best_val_roi = val_roi
        best_params = params
        best_model = model

# Save the best model weights and configuration
if best_model is not None:
    torch.save(best_model.state_dict(), os.path.join("best_models", "best_model_weights2.pth"))
    with open(os.path.join("best_models", "best_config.txt"), "w") as f:
        f.write(f"Best Configuration:\n{best_params}\n")
        f.write(f"Best Validation ROI: {best_val_roi:.4%}\n")

print(f"Best Configuration: {best_params}")
print(f"Best Validation ROI: {best_val_roi:.4%}")

Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.5186171947440932), 'embedding_hidden_layer_sizes': [16], 'embedding_size': 16, 'hidden_layer_sizes': [64, 32, 16], 'learning_rate': np.float64(0.0015751320499779737), 'optimizer': 'Adam', 'weight_decay': np.float64(2.7914686374528733e-06)}
-3.968212789604882 83.94855100316518
Validation ROI for this configuration: -4.7270%
Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.2232334448672798), 'embedding_hidden_layer_sizes': [16, 8], 'embedding_size': 16, 'hidden_layer_sizes': [64, 32], 'learning_rate': np.float64(0.00012966511753760416), 'optimizer': 'SGD', 'weight_decay': np.float64(6.798962421591126e-06)}
-12.335295358804888 141.32701607779916
Validation ROI for this configuration: -8.7282%
Testing configuration: {'batch_size': 64, 'dropout': np.float64(0.20031150633640574), 'embedding_hidden_layer_sizes': [32], 'embedding_size': 4, 'hidden_layer_sizes': [128], 'learning_rate': np.float64(0.000405961161

    ### Zostawiamy tylko CO kolumny

In [38]:
list(matches.columns)

['outdoor',
 'player1_Sets_Before',
 'player1_Gems_Before',
 'player2_Sets_Before',
 'player2_Gems_Before',
 'tournament_level',
 'player1_right_handed',
 'player1_age',
 'player2_right_handed',
 'player2_age',
 'best_of',
 'player1_rank',
 'player2_rank',
 'player1_entry_LL',
 'player1_entry_Q',
 'player1_entry_WC',
 'player2_entry_LL',
 'player2_entry_Q',
 'player2_entry_WC',
 'player1_is_seeded',
 'player2_is_seeded',
 'player1_h2h_wins',
 'player2_h2h_wins',
 'player1_home',
 'player2_home',
 'Round_Num',
 'player1_win_pct_last_10',
 'player2_win_pct_last_10',
 'player1_win_pct_last_10_surface',
 'player2_win_pct_last_10_surface',
 'player1_elo',
 'player2_elo',
 'player1_surface_elo',
 'player2_surface_elo',
 'player1_blended_elo',
 'player2_blended_elo',
 'num_CO_matches',
 'player1_CO_1st_serve_in_pct_avg',
 'player2_CO_1st_serve_in_pct_avg',
 'player1_1st_serve_in_pct_avg',
 'player2_1st_serve_in_pct_avg',
 'player1_CO_1st_serve_win_pct_avg',
 'player2_CO_1st_serve_win_pct_avg'

In [44]:
player1_cols = [col for col in matches.columns if col.startswith('player1_') and not ("avg" in col and "CO" not in col)]
player2_cols = [col for col in matches.columns if col.startswith('player2_') and not ("avg" in col and "CO" not in col)]
env_cols = [col for col in matches.columns if col not in player1_cols and col not in player2_cols and "diff" not in col and "CO" not in col and "target" not in col ]
target_col = 'target'
player1_cols

['player1_Sets_Before',
 'player1_Gems_Before',
 'player1_right_handed',
 'player1_age',
 'player1_rank',
 'player1_entry_LL',
 'player1_entry_Q',
 'player1_entry_WC',
 'player1_is_seeded',
 'player1_h2h_wins',
 'player1_home',
 'player1_win_pct_last_10',
 'player1_win_pct_last_10_surface',
 'player1_elo',
 'player1_surface_elo',
 'player1_blended_elo',
 'player1_CO_1st_serve_in_pct_avg',
 'player1_CO_1st_serve_win_pct_avg',
 'player1_CO_2nd_serve_win_pct_avg',
 'player1_CO_serve_games_win_pct_avg',
 'player1_CO_ace_avg',
 'player1_CO_df_avg',
 'player1_CO_1st_serve_return_win_pct_avg',
 'player1_fatigue_score',
 'player1_tournament_wins_before',
 'player1_tournament_losses_before']

In [45]:
player1_features = matches[player1_cols].values  # Convert to numpy array
player2_features = matches[player2_cols].values
env_features = matches[env_cols].values
match_outcomes = matches[target_col].values  # Target (1 for player1 win, 0 for player2 win)

In [46]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
player1_features = scaler.fit_transform(player1_features)
player2_features = scaler.transform(player2_features)  
env_features = scaler.fit_transform(env_features)

In [47]:
import torch

player1_features = torch.tensor(player1_features, dtype=torch.float32)
player2_features = torch.tensor(player2_features, dtype=torch.float32)
env_features = torch.tensor(env_features, dtype=torch.float32)
match_outcomes = torch.tensor(match_outcomes, dtype=torch.float32).unsqueeze(1)  # Add dimension for compatibility

In [48]:
from sklearn.model_selection import train_test_split

player1_train, player1_test, player2_train, player2_test, env_train, env_test, y_train, y_test = train_test_split(
    player1_features, player2_features, env_features, match_outcomes, test_size=0.2, random_state=42, stratify=match_outcomes
)
player1_train, player1_val, player2_train, player2_val, env_train, env_val, y_train, y_val = train_test_split(
    player1_train, player2_train, env_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
y_val.unique(return_counts=True)

(tensor([0., 1.]), tensor([1129, 1129]))

In [50]:
# Create DataLoader for training
train_dataset = TensorDataset(player1_train, player2_train, env_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Validation DataLoader
val_dataset = TensorDataset(player1_val, player2_val, env_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [52]:
from sklearn.model_selection import ParameterSampler
import scipy.stats as stats
set_seed(42)

# Parameter grid for random search
param_grid = {
    'learning_rate': stats.loguniform(1e-5, 1e-2),
    'dropout': stats.uniform(0.1, 0.6),
    'hidden_layer_sizes': [
       [256], [256, 256], [512], [512,256], [128], [128, 128], [256, 128], [256, 128, 64], [512, 256, 128], [64], [32], [16, 8], [64, 32], [64, 64], [64, 32, 16],
    ],
    'batch_size': [32, 64, 128],
    'weight_decay': stats.loguniform(1e-6, 1e-3),
    'optimizer': ['Adam', 'SGD']
}

# Generate random hyperparameter samples
n_samples = 200
random_params = list(ParameterSampler(param_grid, n_iter=n_samples, random_state=42))

# Training and evaluation function
def train_and_evaluate(params):
    print(f"Testing configuration: {params}")
    
    # Extract parameters
    learning_rate = params['learning_rate']
    dropout = params['dropout']
    hidden_layer_sizes = params['hidden_layer_sizes']
    batch_size = params['batch_size']
    weight_decay = params['weight_decay']
    optimizer_choice = params['optimizer']
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Model
    model = SymmetricNNWithEnvInteraction(
        player_feature_size=player1_train.shape[1],
        env_feature_size=env_train.shape[1],
        embedding_size=8,
        hidden_sizes=hidden_layer_sizes,
        dropout=dropout
    )
    
    # Initialize weights
    def initialize_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.zeros_(m.bias)
    model.apply(initialize_weights)

    # Optimizer
    if optimizer_choice == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_choice == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_choice == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


    # Loss function
    criterion = nn.BCELoss()

     # Training loop
    best_val_accuracy = 0.0
    for epoch in range(20):
        model.train()
        for p1, p2, env, labels in train_loader:
            optimizer.zero_grad()
            predictions = model(p1, p2, env)
            loss = criterion(predictions, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for p1, p2, env, labels in val_loader:
                predictions = model(p1, p2, env)
                predicted = (predictions > 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        val_accuracy = correct / total
        best_val_accuracy = max(best_val_accuracy, val_accuracy)

    print(f"Best Validation Accuracy for this configuration: {best_val_accuracy:.4f}")
    return best_val_accuracy

# Run random search
best_params = None
best_val_accuracy = 0.0

for params in random_params:
    val_accuracy = train_and_evaluate(params)
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

print(f"Best Configuration: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.5779257921161397), 'hidden_layer_sizes': [64, 32, 16], 'learning_rate': np.float64(0.001570297088405539), 'optimizer': 'Adam', 'weight_decay': np.float64(6.173770394704566e-05)}
Best Validation Accuracy for this configuration: 0.6603
Testing configuration: {'batch_size': 64, 'dropout': np.float64(0.1935967122017216), 'hidden_layer_sizes': [32], 'learning_rate': np.float64(0.00023864188780056055), 'optimizer': 'Adam', 'weight_decay': np.float64(6.358358856676247e-05)}
Best Validation Accuracy for this configuration: 0.6506
Testing configuration: {'batch_size': 128, 'dropout': np.float64(0.11235069657748148), 'hidden_layer_sizes': [256, 256], 'learning_rate': np.float64(0.0014655354118727707), 'optimizer': 'SGD', 'weight_decay': np.float64(4.335281794951567e-06)}
Best Validation Accuracy for this configuration: 0.6479
Testing configuration: {'batch_size': 32, 'dropout': np.float64(0.47048890577662994), 'hidden_layer_sizes

In [53]:
from sklearn.model_selection import ParameterSampler
import scipy.stats as stats
set_seed(42)

# Parameter grid for random search
param_grid = {
    'learning_rate': stats.loguniform(1e-4, 1e-2),
    'dropout': stats.uniform(0.2, 0.2),
    'hidden_layer_sizes': [
        [128], [256, 128], [64], [32], [16, 8], [64, 32], [64, 64], [64, 32, 16], [32,32], [128,128], [64,16]
    ],
    'batch_size': [32, 64],
    'weight_decay': stats.loguniform(1e-6, 1e-5),
    'optimizer': ['Adam', 'SGD'],
    'embedding_size': [4,8,16],
    'embedding_hidden_layer_sizes': [[128], [256, 128], [64], [32], [16, 8], [64, 32], [16]]
}

# Generate random hyperparameter samples
n_samples = 200
random_params = list(ParameterSampler(param_grid, n_iter=n_samples, random_state=42))

# Training and evaluation function
def train_and_evaluate(params):
    print(f"Testing configuration: {params}")
    
    # Extract parameters
    learning_rate = params['learning_rate']
    dropout = params['dropout']
    hidden_layer_sizes = params['hidden_layer_sizes']
    batch_size = params['batch_size']
    weight_decay = params['weight_decay']
    optimizer_choice = params['optimizer']
    embedding_size = params['embedding_size']
    embedding_hidden_layer_sizes = params['embedding_hidden_layer_sizes']
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Model
    model = SymmetricNNWithEnvInteraction(
        player_feature_size=player1_train.shape[1],
        env_feature_size=env_train.shape[1],
        embedding_size=embedding_size,
        hidden_sizes=hidden_layer_sizes,
        dropout=dropout,
        embedding_hidden_sizes=embedding_hidden_layer_sizes
    )
    
    # Initialize weights
    def initialize_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.zeros_(m.bias)
    model.apply(initialize_weights)

    # Optimizer
    if optimizer_choice == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_choice == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_choice == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


    # Loss function
    criterion = nn.BCELoss()

     # Training loop
    best_val_accuracy = 0.0
    for epoch in range(20):
        model.train()
        for p1, p2, env, labels in train_loader:
            optimizer.zero_grad()
            predictions = model(p1, p2, env)
            loss = criterion(predictions, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for p1, p2, env, labels in val_loader:
                predictions = model(p1, p2, env)
                predicted = (predictions > 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        val_accuracy = correct / total
        best_val_accuracy = max(best_val_accuracy, val_accuracy)

    print(f"Best Validation Accuracy for this configuration: {best_val_accuracy:.4f}")
    return best_val_accuracy

# Run random search
best_params = None
best_val_accuracy = 0.0

for params in random_params:
    val_accuracy = train_and_evaluate(params)
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

print(f"Best Configuration: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

Testing configuration: {'batch_size': 32, 'dropout': np.float64(0.35930859737204657), 'embedding_hidden_layer_sizes': [16], 'embedding_size': 16, 'hidden_layer_sizes': [64, 32, 16], 'learning_rate': np.float64(0.0015751320499779737), 'optimizer': 'Adam', 'weight_decay': np.float64(2.7914686374528733e-06)}
Best Validation Accuracy for this configuration: 0.6532
Testing configuration: {'batch_size': 32, 'dropout': np.float64(0.2116167224336399), 'embedding_hidden_layer_sizes': [16, 8], 'embedding_size': 16, 'hidden_layer_sizes': [64, 32], 'learning_rate': np.float64(0.00012966511753760416), 'optimizer': 'SGD', 'weight_decay': np.float64(6.798962421591126e-06)}
Best Validation Accuracy for this configuration: 0.5996
Testing configuration: {'batch_size': 64, 'dropout': np.float64(0.2001557531682029), 'embedding_hidden_layer_sizes': [32], 'embedding_size': 4, 'hidden_layer_sizes': [128], 'learning_rate': np.float64(0.0004059611610484307), 'optimizer': 'SGD', 'weight_decay': np.float64(1.016